# Reproducing table 6

This notebook currently imports results from a run of PHC.py and attempts to map them to Table 6.

Table 6 results are from Shoaib M, Ramamohan V. **Simulation modeling and analysis of primary health center operations**. *SIMULATION* 98(3):183-208. (2022). <https://doi.org/10.1177/00375497211030931>.

## Set up

In [ ]:
# To run model
import PHC
import numpy as np

# To import and process results
import pandas as pd
import xlrd
import os

# Additional package to record runtime of this notebook
import time
start = time.time()

In [ ]:
table6_path = '../original_study/tab6.csv'

## Run model

Set parameters for each configuration based on Table 3.

In [ ]:
# For all: days 365, warm up 180, 6 inpatient beds, 1 delivery
# bed, 3 staff nurse, 1 NCD nurse. These are default parameters in the model,
# and below, have just specified the parameters that changed between
# configurations

t6_c1_param = {
    'doc_cap': 2,
    'OPD_iat': 4,
    'IPD_iat': 2880,
    'delivery_iat': 1440,
    'ANC_iat': 1440,
    'rep_file': 't6_c1.xls',
    'replication': 100
}

t6_c2_param = {
    'doc_cap': 1,
    'OPD_iat': 9,
    'IPD_iat': 2880,
    'delivery_iat': 2880,
    'ANC_iat': 2880,
    'rep_file': 't6_c2.xls',
    'replication': 100
}

t6_c3_param = {
    'doc_cap': 1,
    'OPD_iat': 9,
    'IPD_iat': 2880,
    'any_delivery': False,
    'any_ANC': False,
    'rep_file': 't6_c3.xls',
    'replication': 100
}

t6_c4_param = {
    'doc_cap': 2,
    'OPD_iat': 3,
    'mean': 5,
    'sd': 1,
    'consult_boundary_1': 2,
    'consult_boundary_2': 2,
    'IPD_iat': 2880,
    'delivery_iat': 1440,
    'ANC_iat': 1440,
    'rep_file': 't6_c4.xls',
    'replication': 100
}

Run model for each configuration.

In [ ]:
# Input configuration parameters to main() (have to append with 's_', and
# add ** as inputting dict as parameters)
for config in [t6_c1_param, t6_c2_param, t6_c3_param, t6_c4_param]:
    print(f'''Running: {config['rep_file']}''')
    PHC.main(**{f's_{k}': v for k, v in config.items()})

## Import and process replication results

In [ ]:
# Make dictionary with labels from table 6, and corresponding names from model output
t6_labels = {
  'doc occ': 'Doctor utilisation',
  'NCD occ': 'NCD Nurse utilisation',
  'staff nurse occ': 'Staff nurse utilisation',
  'pharm occ': 'Pharmacist utilisation',
  'lab occ': 'Lab utilisation',
  'ipd bed occ': 'Inpatient bed utilisation',
  'del occ': 'Labour bed utilisation',  # "Del" stands for delivery
  'opd q len': 'Mean length of OPD queue (number of patients)',
  'OPD Q wt': 'OPD queue waiting time (minutes)',
  'pharmacy q len': 'Mean length of pharmacy queue (number of patients)',
  'Pharmacy Q wt': 'Pharmacy queue waiting time (minutes)',
  'lab q len': 'Mean length of Lab queue (number of patients)',
  'Lab Q wt': 'Lab queue waiting time (minutes)',
  'prop_del_referred': 'Fraction of childbirth cases referred'
}

In [ ]:
# List of files to loop through
files = ['t6_c1', 't6_c2', 't6_c3', 't6_c4']

# Empty list to store results
result_list = []

for f in files:
    # Import .xls and convert to pandas dataframe
    book = xlrd.open_workbook(os.path.join('outputs', f'{f}.xls'))
    result = pd.read_excel(book, header=None, index_col=0)

    # Add proportion of childbirth cases referred
    result.loc['prop_del_referred'] = (
        result.loc['del referred'] / result.loc['Del patients'])

    # Find mean and standard deviation from the replication
    # Save as dataframe, dropping the duplicate rows (NCD occ twice)
    res = pd.DataFrame({
        f'model_{f}_mean': result.mean(axis=1),
        f'model_{f}_sd': result.std(axis=1)
    }).drop_duplicates()

    # Save to list
    result_list.append(res)

In [ ]:
# Combine into single dataframe
summary = (pd.concat(result_list, axis=1)
           .reset_index()
           .rename(columns= {0: 'model_outcome'}))

# Add labels to model results
summary['t6_outcome'] = summary['model_outcome'].map(t6_labels)

summary

## Import table 6 results and compare against run results

In [ ]:
# Import table 6
t6 = pd.read_csv(table6_path).rename(columns={'outcome': 't6_outcome'})
t6

In [ ]:
# Merge
compare = t6.merge(summary)
compare.head()

In [ ]:
compare_col = [
    ('config1_mean', 'model_t6_c1_mean'),
    ('config1_sd', 'model_t6_c1_sd'),
    ('config2_mean', 'model_t6_c2_mean'),
    ('config2_sd', 'model_t6_c2_sd'),
    ('config3_mean', 'model_t6_c3_mean'),
    ('config3_sd', 'model_t6_c3_sd'),
    ('benchmark_mean', 'model_t6_c4_mean'),
    ('benchmark_sd', 'model_t6_c4_sd'),]

for col in compare_col:
    # Find difference between two columns
    compare[f'change_{col[1]}'] = abs(compare[col[1]] - compare[col[0]])
    # Find percent change between two columns
    subset = compare[list(col)]
    pct_change = subset.pct_change(axis=1).iloc[:, 1]*100
    compare[f'pct_change_{col[1]}'] = pct_change

# Display each of the results
for col in compare_col:
    # Set outcome as index, and get the two results plus percent change
    subset = compare.set_index('t6_outcome')[
        list(col) + [f'change_{col[1]}', f'pct_change_{col[1]}']]
    display(subset)

## Format model results like table 6, and save

In [ ]:
compare_col = [
    ['Configuration 1', 'model_t6_c1_mean', 'model_t6_c1_sd'],
    ['Configuration 2', 'model_t6_c2_mean', 'model_t6_c2_sd'],
    ['Configuration 3', 'model_t6_c3_mean', 'model_t6_c3_sd'],
    ['Benchmark Case', 'model_t6_c4_mean', 'model_t6_c4_sd']]

# Combine each pair of columns into string as in Table 6
for col in compare_col:
    compare[col[0]] = (
        round(compare[col[1]], 3).astype(str) + ' (' +
        round(compare[col[2]], 3).astype(str) + ')')

# Set outcome name as index, and select those columns
formatted_model_res = (compare
                       .set_index('t6_outcome')
                       .rename_axis('Simulation Outcome')[
                           [item[0] for item in compare_col]])
formatted_model_res

In [ ]:
# Find run time in seconds
end = time.time()
runtime = round(end-start)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime//60}m {runtime%60}s')